<a href="https://colab.research.google.com/github/vaniamv/SLU05/blob/main/streaming_ELT__DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Aplication in Real Time to Read Carris API - group 1 - ELT approach

This notebook documents the steps to implement a data pipeline leveraging Google Cloud Platform (GCP), following an **ELT (Extract, Load, Transform)** approach. The pipeline processes data in two stages:


1.   *Streaming Ingestion (Extract and Load):*
Data is ingested in real-time from a source bucket on GCP (input bucket) and stored in a bronze layer bucket, preserving the raw format for further processing.
2.   *Batch Transformation (Transform):*
The raw data from the bronze layer is transformed in batches. These transformations clean, standardize, and structure the data, preparing it for analytical use. The transformed data is then stored in the silver layer bucket for downstream consumption.

By prioritizing the ELT approach, this pipeline ensures flexibility in processing and allows the raw data to remain available for future transformations, ensuring adaptability to evolving business requirements.


---



---








1. Authentication to Google Cloud Platform (GCP)



In [ ]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=a0kvIrXqgfbvH62vbjYYtmBdJjN1sy&prompt=consent&token_usage=remote&access_type=offline&code_challenge=OhHjzMPMajihRcurFDe22Y6GY9j7_BCxQqEWrHMjiOY&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0ASVgi3KHYeCrU5QNe1r-bp-dg4d_swE1CXdMW6YO_Vj26udcEP22PMX5U1pk0wZpj8S7MQ

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [ ]:
# download connector and save it local

!wget https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.7/gcs-connector-hadoop3-2.2.7-shaded.jar -P /usr/local/lib/

--2025-01-23 10:01:46--  https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.7/gcs-connector-hadoop3-2.2.7-shaded.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33831577 (32M) [application/java-archive]
Saving to: ‘/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar’

gcs-connector-hadoo 100%[===================>]  32.26M   147MB/s    in 0.2s    

2025-01-23 10:01:47 (147 MB/s) - ‘/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar’ saved [33831577/33831577]





---


---
2. Initialize SparkSession and set up the access to GSC


In [ ]:
# import libraries

import os
from pyspark.sql import SparkSession

#spark session
spark = SparkSession.builder \
    .appName('GCS_Spark') \
    .config('spark.jars', '/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar') \
    .config('spark.hadoop.fs.gs.impl', 'com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem') \
    .getOrCreate()

# save credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/content/.config/application_default_credentials.json'

# Config PySpark to access the GCS
spark._jsc.hadoopConfiguration().set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
spark._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
spark._jsc.hadoopConfiguration().set("fs.gs.auth.service.account.enable", "true")
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile", '/content/.config/application_default_credentials.json')




---


---
3. Set up the source schema and initialize the readStream

In [ ]:
from pyspark.sql.types import *

# create schema
vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                             StructField('block_id', StringType(), True),
                             StructField('current_status', StringType(), True),
                             StructField('id', StringType(), True),
                             StructField('lat', FloatType(), True),
                             StructField('line_id', StringType(), True),
                             StructField('lon', FloatType(), True),
                             StructField('pattern_id', StringType(), True),
                             StructField('route_id', StringType(), True),
                             StructField('schedule_relationship', StringType(), True),
                             StructField('shift_id', StringType(), True),
                             StructField('speed', FloatType(), True),
                             StructField('stop_id', StringType(), True),
                             StructField('timestamp', TimestampType(), True),
                             StructField('trip_id', StringType(), True)])


#readStreaming
stream = spark.readStream.format("json").schema(vehicle_schema).load("gs://edit-de-project-streaming-data/carris-vehicles")

In [ ]:
#confirm that stream is streaming
stream.isStreaming

True



---


---
4. Write the stream in a bronze layer (landing zone)
* Purpose: Raw data ingestion layer.
* Data Characteristics: Raw, unprocessed, and schema-on-read where feasible.
* Data Storage: Store data exactly as ingested (in this case JSON format).
* Operations: Minimal transformation; only schema enforcement and deduplication.

In [ ]:
bronze_layer="gs://edit-data-eng-project-group1/datalake/stream/ELT/bronze_layer"

# writeStreaming
query = (stream
        .writeStream
        .outputMode("append")
        .option("path", bronze_layer)
        .option('checkpointLocation', 'gs://edit-data-eng-project-group1/datalake/stream/ELT/bronze_layer/checkpoint')
        .start()

        )

query.awaitTermination(60)

False

In [ ]:
#check the status of the query
query.status

{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

In [ ]:
#check if the write streaming is active
query.isActive

True

In [ ]:
query.stop()



---


---
5. Check if the bronze layer received the files and prepare the bronze layer operations (schema enforcement and deduplication)

In [ ]:
vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                             StructField('block_id', StringType(), True),
                             StructField('current_status', StringType(), True),
                             StructField('id', StringType(), True),
                             StructField('lat', FloatType(), True),
                             StructField('line_id', StringType(), True),
                             StructField('lon', FloatType(), True),
                             StructField('pattern_id', StringType(), True),
                             StructField('route_id', StringType(), True),
                             StructField('schedule_relationship', StringType(), True),
                             StructField('shift_id', StringType(), True),
                             StructField('speed', FloatType(), True),
                             StructField('stop_id', StringType(), True),
                             StructField('timestamp', TimestampType(), True),
                             StructField('trip_id', StringType(), True)])

# Read the Parquet files into a DataFrame
parquet_df = spark.read.schema(vehicle_schema).parquet(bronze_layer)

#parquet_df = parquet_df.drop_duplicates()

# Show the first few rows
parquet_df.show(truncate=False)

# Print the schema to understand the data structure
parquet_df.printSchema()


+-------+------------------------------+--------------+--------+---------+-------+----------+----------+--------+---------------------+------------+---------+-------+-------------------+------------------------+
|bearing|block_id                      |current_status|id      |lat      |line_id|lon       |pattern_id|route_id|schedule_relationship|shift_id    |speed    |stop_id|timestamp          |trip_id                 |
+-------+------------------------------+--------------+--------+---------+-------+----------+----------+--------+---------------------+------------+---------+-------+-------------------+------------------------+
|137    |20250121-64010205-112450234560|IN_TRANSIT_TO |44|12094|38.513027|4401   |-8.8591585|4401_0_2  |4401_0  |SCHEDULED            |112450234560|3.6111112|160479 |2025-01-21 08:42:12|4401_0_2|3200|0830_AEMLZ|
|311    |20250121-64010028-123290234560|IN_TRANSIT_TO |44|13597|38.701008|4705   |-8.9522085|4705_0_2  |4705_0  |SCHEDULED            |123290234560|4.72



---


---
6. Ingest to the silver layer
* Purpose: Cleaned and enriched data layer.
* Data Characteristics: Schema-on-write, normalized/structured, with quality checks applied.
* Operations:
Filter for getting only the columns required.
Joins with reference data (Historical STOPS) for enrichment.
Simple calculations and store it in new columns.

In [ ]:
from pyspark.sql.functions import lag , col, coalesce
from pyspark.sql.window import Window
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
import math

def haversine_distance(lat1, lon1, lat2, lon2):

    if any(x is None for x in [lat1, lon1, lat2, lon2]):
        return 0.0
    R = 6371  # Earth's radius in kilometers

    # Convert latitude and longitude to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Calculate differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Apply Haversine formula
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))

    # Calculate distance
    distance = R * c

    return distance

# Register the UDF
distance_udf = udf(haversine_distance, FloatType())

# Define a window specification
windowSpec = Window.partitionBy("id").orderBy("timestamp")

#select columns
transform = parquet_df.select('id', 'speed', 'timestamp','line_id','route_id','stop_id','lat', 'lon')

# Create a new column 'previous_value' using lag
transform = transform.withColumn("previous_lat", coalesce(lag("lat", 1).over(windowSpec), col('lat')))
transform = transform.withColumn("previous_lon", coalesce(lag("lon", 1).over(windowSpec), col('lon')))

# Get the dataset from endpoint STOPS that we need to join to our main dataset
df_stops = spark.read.option("header", "true").csv('gs://edit-data-eng-project-group1/LandingZone/GTFS/stops.txt')
df_stops = df_stops.select('stop_id','stop_lat','stop_lon')
df_stops = df_stops.withColumn("stop_lat", df_stops["stop_lat"].cast("float"))
df_stops = df_stops.withColumn("stop_lon", df_stops["stop_lon"].cast("float"))

# Join and add new calculated columns
transform = transform.join(df_stops, on='stop_id', how='left')

transform = transform.withColumn("distance", distance_udf(transform["previous_lat"],transform["previous_lon"],transform["lat"],transform["lon"]))
transform = transform.withColumn("distance_to_stop", distance_udf(transform["lat"],transform["lon"],transform["stop_lat"],transform["stop_lon"]))

transform.show()

+-------+-------+---------+-------------------+-------+--------+---------+---------+------------+------------+---------+---------+-----------+----------------+
|stop_id|     id|    speed|          timestamp|line_id|route_id|      lat|      lon|previous_lat|previous_lon| stop_lat| stop_lon|   distance|distance_to_stop|
+-------+-------+---------+-------------------+-------+--------+---------+---------+------------+------------+---------+---------+-----------+----------------+
| 120279|41|1100|0.2777778|2025-01-21 08:26:45|   1606|  1606_0| 38.72538|-9.246014|    38.72538|   -9.246014|38.725388|-9.246036|        0.0|    0.0020833486|
| 120281|41|1100| 6.111111|2025-01-21 08:27:23|   1606|  1606_0|38.725426|-9.246371|    38.72538|   -9.246014|38.725094|-9.249104| 0.03143868|      0.23987857|
| 120281|41|1100| 6.111111|2025-01-21 08:27:23|   1606|  1606_0|38.725426|-9.246371|   38.725426|   -9.246371|38.725094|-9.249104|        0.0|      0.23987857|
| 120431|41|1100|3.3333333|2025-01-21 08

In [ ]:
transform.printSchema()

root
 |-- stop_id: string (nullable = true)
 |-- id: string (nullable = true)
 |-- speed: float (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- line_id: string (nullable = true)
 |-- route_id: string (nullable = true)
 |-- lat: float (nullable = true)
 |-- lon: float (nullable = true)
 |-- previous_lat: float (nullable = true)
 |-- previous_lon: float (nullable = true)
 |-- stop_lat: float (nullable = true)
 |-- stop_lon: float (nullable = true)
 |-- distance: float (nullable = true)
 |-- distance_to_stop: float (nullable = true)



In [ ]:
transform.write.format("parquet").mode("overwrite").save("gs://edit-data-eng-project-group1/datalake/stream/ELT/silver_layer")



---


---
7. Ingest to the gold layer
* Purpose: Aggregated data ready for analytics and reporting.
* Data Characteristics: Pre-aggregated, aggregated by business logic, optimized for query performance.
* Operations: Aggregations, windowed calculations. Metrics computations (e.g., averages, counts, sums).

In [ ]:
transform = spark.read.format("parquet").load("gs://edit-data-eng-project-group1/datalake/stream/ELT/silver_layer")

In [ ]:
import pyspark.sql.functions as F


agg = transform.groupBy("id", "stop_id", F.window("timestamp", "2 minutes")).agg(
    F.sum('distance').alias("distance_2_min"),
    F.last('distance_to_stop').alias('distance_to_stop')
    )

agg = agg.withColumn('speed', col('distance_2_min')/(2/60))

agg = agg.filter(agg.distance_to_stop.isNotNull() & (agg.distance_to_stop > 0) & (agg.speed.isNotNull()) & (agg.speed > 0)) \
         .withColumn('time_to_stop', (col('distance_to_stop')/col('speed') * 3600))

agg = agg.withColumn(
    'time_to_stop',
    F.from_unixtime(
        F.unix_timestamp(F.lit('00:00:00'), 'HH:mm:ss') + col('time_to_stop'),
        'HH:mm:ss'
    )
)


agg.show()

+-------+-------+--------------------+-------------------+----------------+------------------+------------+
|     id|stop_id|              window|     distance_2_min|distance_to_stop|             speed|time_to_stop|
+-------+-------+--------------------+-------------------+----------------+------------------+------------+
|41|1102| 120033|{2025-01-21 09:04...|0.28879860043525696|     0.010783878| 8.663958013057709|    00:00:04|
|41|1102| 120283|{2025-01-21 09:14...| 0.2967355474829674|    0.0025463942| 8.902066424489021|    00:00:01|
|41|1102| 120261|{2025-01-22 07:12...|0.10032915696501732|      0.03582338|3.0098747089505196|    00:00:42|
|41|1102| 120181|{2025-01-22 17:20...| 0.3366996943950653|     0.006795796| 10.10099083185196|    00:00:02|
|41|1102| 120261|{2025-01-23 07:12...|0.06684991717338562|     0.002651806|2.0054975152015686|    00:00:04|
|41|1102| 120061|{2025-01-23 11:00...| 0.1589718833565712|    0.0050084856| 4.769156500697136|    00:00:03|
|41|1103| 120015|{2025-01-21

In [ ]:
transform.write.format("parquet").mode("overwrite").save("gs://edit-data-eng-project-group1/datalake/stream/ELT/gold_layer")